In [4]:
import pandas as pd, numpy as np

train_df = pd.read_csv('data/playground-series-s5e11/train.csv')
test_df = pd.read_csv('data/playground-series-s5e11/test.csv')
orig = pd.read_csv('data/playground-series-s5e11/loan_dataset_20000.csv')

print('Train Shape:', train_df.shape)
print('Test Shape:', test_df.shape)

Train Shape: (593994, 13)
Test Shape: (254569, 12)


In [7]:
train_df.head

<bound method NDFrame.head of             id  annual_income  debt_to_income_ratio  credit_score  \
0            0       29367.99                 0.084           736   
1            1       22108.02                 0.166           636   
2            2       49566.20                 0.097           694   
3            3       46858.25                 0.065           533   
4            4       25496.70                 0.053           665   
...        ...            ...                   ...           ...   
593989  593989       23004.26                 0.152           703   
593990  593990       35289.43                 0.105           559   
593991  593991       47112.64                 0.072           675   
593992  593992       76748.44                 0.067           740   
593993  593993       48959.52                 0.096           752   

        loan_amount  interest_rate  gender marital_status education_level  \
0           2528.42          13.67  Female         Single     Hi

In [5]:
TARGET = 'loan_paid_back'  #boolean
CATS = ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']
BASE = [col for col in train_df.columns if col not in ['id', TARGET]]

train = train_df.copy()
test = test_df.copy()

In [6]:
ORIG = []

for col in BASE:
    mean_map = orig.groupby(col)[TARGET].mean()
    new_mean_col_name = f"orig_mean_{col}"
    mean_map.name = new_mean_col_name
    
    train = train.merge(mean_map, on=col, how='left')
    test = test.merge(mean_map, on=col, how='left')
    ORIG.append(new_mean_col_name)

    new_count_col_name = f"orig_count_{col}"
    count_map = orig.groupby(col).size().reset_index(name=new_count_col_name)
    
    train = train.merge(count_map, on=col, how='left')
    test = test.merge(count_map, on=col, how='left')
    ORIG.append(new_count_col_name)

print(len(ORIG), 'Orig Features Created!!')

FEATURES = BASE + ORIG
print(len(FEATURES), 'Total Features.')

22 Orig Features Created!!
33 Total Features.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train[FEATURES], train[TARGET].astype(int), 
    test_size=0.2, random_state=42, stratify=train[TARGET]
)

print(f'Train split (with ORIG features): {X_train.shape}')
print(f'Val split (with ORIG features): {X_val.shape}')